In [1]:
# Extração da média de cargas: Coulomb e LJ
# B1 - PS

avogrado = 6.02e22

print('B1-PS')
print(round(2.7331632e+22/avogrado,4),'eV - Coulomb')
print(round(-4.0678101e23/avogrado,4),'eV - LJ')

print('\nCBM-Spid')
print(round(-2.449006e+24/avogrado,4),'eV - Coulomb')
print(round(-7.0256926e+23/avogrado,4),'eV - LJ')

print('\nSpid-PS')
print(round(2.95235251e+23/avogrado,4),'eV - Coulomb')
print(round(-5.1449448e+22/avogrado,4),'eV - LJ')

print('\nB1-B1')
print(round(4.9752819e+22/avogrado,4),'ev - Coulomb')
print(round(2.3915154e+23/avogrado,4),'ev - LJ')

print('\nSpid-Spid')
print(round(1.1105705e+24/avogrado,4),'ev - Coulomb')
print(round(4.70770211e+24/avogrado,4),'ev - LJ')

print('\nPS-PS')
# print(round(
# print(round(

B1-PS
0.454 eV - Coulomb
-6.7572 eV - LJ

CBM-Spid
-40.6812 eV - Coulomb
-11.6706 eV - LJ

Spid-PS
4.9042 eV - Coulomb
-0.8546 eV - LJ

B1-B1
0.8265 ev - Coulomb
3.9726 ev - LJ

Spid-Spid
18.448 ev - Coulomb
78.201 ev - LJ

PS-PS
